## Script de génération des données :

In [10]:
import random
import string
import json

# Générer des données de livres
def generate_books(num_books):
    books = []
    for i in range(num_books):
        book = {
            "title": ''.join(random.choices(string.ascii_uppercase + string.digits, k=10)),
            "author_id": f"author{random.randint(1, 100)}",
            "published_year": random.randint(1900, 2023)
        }
        books.append(book)
    return books

# Générer des données d'auteurs
def generate_authors(num_authors):
    authors = []
    for i in range(num_authors):
        author = {
            "name": ''.join(random.choices(string.ascii_uppercase + string.digits, k=10)),
            "birth_year": random.randint(1900, 2000)
        }
        authors.append(author)
    return authors

# Générer des données d'emprunts
def generate_borrows(num_borrows):
    borrows = []
    for i in range(num_borrows):
        borrow = {
            "book_id": f"book{random.randint(1, 1000)}",
            "borrow_date": f"2023-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}",
            "return_date": f"2023-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}"
        }
        borrows.append(borrow)
    return borrows

# Sauvegarder les données générées dans des fichiers JSON
num_books = 10000
num_authors = 1000
num_borrows = 5000

books = generate_books(num_books)
authors = generate_authors(num_authors)
borrows = generate_borrows(num_borrows)

with open('data/books.json', 'w') as f:
    json.dump(books, f)

with open('data/authors.json', 'w') as f:
    json.dump(authors, f)

with open('data/borrows.json', 'w') as f:
    json.dump(borrows, f)


## Scripts pour importer les données et mesurer les performances :


### Les Performance de CouchDB

In [13]:
import time
import requests

# Configuration de CouchDB
couchdb_url = 'http://127.0.0.1:5984/'
couchdb_auth = ('Almamy', '00000000')
db_name = 'library'

# Fonction pour mesurer le temps d'exécution
def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f'Time taken: {end_time - start_time} seconds')
        return result
    return wrapper

# Création de la base de données
requests.put(f'{couchdb_url}{db_name}', auth=couchdb_auth)

# Fonction pour ajouter des documents
def add_document(doc, doc_id=None):
    if doc_id:
        response = requests.put(f'{couchdb_url}{db_name}/{doc_id}', auth=couchdb_auth, json=doc)
    else:
        response = requests.post(f'{couchdb_url}{db_name}', auth=couchdb_auth, json=doc)
    return response.json()

# Charger les données
with open('data/books.json', 'r') as f:
    books = json.load(f)
with open('data/authors.json', 'r') as f:
    authors = json.load(f)
with open('data/borrows.json', 'r') as f:
    borrows = json.load(f)

# Ajouter les documents dans CouchDB
@measure_time
def load_data():
    for i, book in enumerate(books):
        add_document(book, f'book{i+1}')
    for i, author in enumerate(authors):
        add_document(author, f'author{i+1}')
    for i, borrow in enumerate(borrows):
        add_document(borrow, f'borrow{i+1}')

load_data()

# Mesurer les performances des opérations CRUD
@measure_time
def create_document(doc, doc_id=None):
    if doc_id:
        response = requests.put(f'{couchdb_url}{db_name}/{doc_id}', auth=couchdb_auth, json=doc)
    else:
        response = requests.post(f'{couchdb_url}{db_name}', auth=couchdb_auth, json=doc)
    return response.json()

@measure_time
def read_document(doc_id):
    response = requests.get(f'{couchdb_url}{db_name}/{doc_id}', auth=couchdb_auth)
    return response.json()

@measure_time
def update_document(doc_id, update):
    doc = read_document(doc_id)
    doc.update(update)
    response = requests.put(f'{couchdb_url}{db_name}/{doc_id}', auth=couchdb_auth, json=doc)
    return response.json()

@measure_time
def delete_document(doc_id):
    doc = read_document(doc_id)
    response = requests.delete(f'{couchdb_url}{db_name}/{doc_id}?rev={doc["_rev"]}', auth=couchdb_auth)
    return response.json()

# Exemple d'utilisation
doc_id = 'book1'
create_document({"title": "New Book", "author_id": "author1", "published_year": 2021}, doc_id)
read_document(doc_id)
update_document(doc_id, {"title": "Updated Book"})
delete_document(doc_id)


Time taken: 313.5557017326355 seconds
Time taken: 0.02392101287841797 seconds
Time taken: 0.018618345260620117 seconds
Time taken: 0.01402139663696289 seconds
Time taken: 0.039955854415893555 seconds
Time taken: 0.02508378028869629 seconds
Time taken: 0.07315921783447266 seconds


{'ok': True, 'id': 'book1', 'rev': '18-b35bff33348ea635ec5d94f7fbd4d8d5'}

### Les Performance de Redis

In [9]:
import time
import redis
import json

# Configuration de Redis
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0)

# Fonction pour mesurer le temps d'exécution
def measure_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f'Time taken: {end_time - start_time} seconds')
        return result
    return wrapper

# Charger les données
with open('data/books.json', 'r') as f:
    books = json.load(f)
with open('data/authors.json', 'r') as f:
    authors = json.load(f)
with open('data/borrows.json', 'r') as f:
    borrows = json.load(f)

# Ajouter les documents dans Redis
@measure_time
def load_data():
    for i, book in enumerate(books):
        redis_client.hmset(f'book{i+1}', book)
    for i, author in enumerate(authors):
        redis_client.hmset(f'author{i+1}', author)
    for i, borrow in enumerate(borrows):
        redis_client.hmset(f'borrow{i+1}', borrow)

load_data()

# Mesurer les performances des opérations CRUD
@measure_time
def create_document(doc, doc_id):
    redis_client.hmset(doc_id, doc)

@measure_time
def read_document(doc_id):
    return redis_client.hgetall(doc_id)

@measure_time
def update_document(doc_id, update):
    redis_client.hmset(doc_id, update)

@measure_time
def delete_document(doc_id):
    redis_client.delete(doc_id)

# Exemple d'utilisation
doc_id = 'book1'
create_document({"title": "New Book", "author_id": "author1", "published_year": 2021}, doc_id)
read_document(doc_id)
update_document(doc_id, {"title": "Updated Book"})
delete_document(doc_id)


C:\Users\HP\AppData\Local\Temp\ipykernel_17580\2885376056.py:30: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  redis_client.hmset(f'book{i+1}', book)
C:\Users\HP\AppData\Local\Temp\ipykernel_17580\2885376056.py:32: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  redis_client.hmset(f'author{i+1}', author)
C:\Users\HP\AppData\Local\Temp\ipykernel_17580\2885376056.py:34: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  redis_client.hmset(f'borrow{i+1}', borrow)


Time taken: 6.314136981964111 seconds
Time taken: 0.0019996166229248047 seconds
Time taken: 0.01465749740600586 seconds
Time taken: 0.0010218620300292969 seconds
Time taken: 0.0 seconds


C:\Users\HP\AppData\Local\Temp\ipykernel_17580\2885376056.py:41: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  redis_client.hmset(doc_id, doc)
C:\Users\HP\AppData\Local\Temp\ipykernel_17580\2885376056.py:49: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  redis_client.hmset(doc_id, update)
